In [4]:
# Import the data from csv file and explore the data

import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
import json

# **** SETUP ****

# change to match your filesystem
DATA_DIR = "./data/"
DEFAULT_RECEIPTS_FILE = os.path.join(DATA_DIR, "World_Energy_Consumption.csv")
PROJECT_NAME = "deb-01-371820"
DATASET_NAME = "emissions"

data = []

df = pd.read_csv(DEFAULT_RECEIPTS_FILE)

display(df.head(n=10))



,iso_code,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,...,solar_elec_per_capita,solar_energy_per_capita,gdp,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
0,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AFG,Afghanistan,1905,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AFG,Afghanistan,1906,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AFG,Afghanistan,1907,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AFG,Afghanistan,1908,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AFG,Afghanistan,1909,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# create the gdf dataframe in order to prep for fct_gdp table creation


# extract 
gdp_df = df
gdp_cols = ['iso_code', 'country', 'year', 'gdp', 'population']

gdp_df = gdp_df[gdp_cols]

gdp_df= gdp_df.rename(columns={
    'iso_code': 'country_code', 
    'country':'country_name', 
    })

def composite_key(row):
    comp_key = row.country_code + row.year
    return comp_key




gdp_df['created_at'] = pd.Timestamp.now()
gdp_df['modified_at'] = None


gdp_df['country_code_year'] = gdp_df.apply(composite_key)

display(gdp_df.head())




AttributeError: 'Series' object has no attribute 'country_code'

In [ ]:
consump_df = df
consump_df = consump_df[['iso_code', 'country', 'year', 'biofuel_consumption', 'coal_consumption','fossil_fuel_consumption', 'gas_consumption','hydro_consumption', 'low_carbon_consumption','nuclear_consumption','oil_consumption','other_renewable_consumption','primary_energy_consumption','renewables_consumption', 'solar_consumption','wind_consumption']]